In [14]:
import pandas as pd
import json

# Load the JSON data from file
with open('2023_f.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [19]:
# Gör om datan till en pandas dataframe
df = pd.DataFrame(data)

df.tail(5)

,id,webpage_url,headline,application_deadline,salary_description,experience_required,driving_license_required,application_contacts,publication_date,source_type,...,must_have.skills,must_have.languages,must_have.work_experiences,must_have.education,must_have.education_level,nice_to_have.skills,nice_to_have.languages,nice_to_have.work_experiences,nice_to_have.education,nice_to_have.education_level
17710,26968624,https://arbetsformedlingen.se/platsbanken/anno...,Blivande NMS Engineers (Junior/team),2023-01-31T23:59:59,None,True,False,Josefine Mellberg,2023-01-01T22:23:58,VIA_PLATSBANKEN_DXA,...,None,None,Nätverksoperatör,None,None,None,None,None,None,None
17711,26968672,https://arbetsformedlingen.se/platsbanken/anno...,Remote: Älskar du Python/Django? Ansök via SSH!,2023-01-31T23:59:59,None,True,False,None,2023-01-01T23:01:34,VIA_ANNONSERA,...,None,None,None,None,None,None,None,None,None,None
17712,26968673,https://arbetsformedlingen.se/platsbanken/anno...,Remote: Älskar du TypeScript & React? Ansök vi...,2023-01-31T23:59:59,None,True,False,None,2023-01-01T23:01:44,VIA_ANNONSERA,...,None,None,None,None,None,None,None,None,None,None
17713,26968683,https://arbetsformedlingen.se/platsbanken/anno...,SW-Engineer - Senior,2023-06-29T23:59:59,None,True,False,Erik Bernesson,2023-01-01T23:36:34,VIA_ANNONSERA,...,"[C, programmeringsspråk, C++ Builder, programm...","[Engelska, Svenska]",Systemutvecklare/Programmerare,Data,Eftergymnasial utbildning två år eller längre,"[C#, programmeringsspråk, CANalyzer, analysver...",None,None,None,None
17714,26968682,https://arbetsformedlingen.se/platsbanken/anno...,SW-Engineer,2023-06-27T23:59:59,None,True,False,Erik Bernesson,2023-01-01T23:35:04,VIA_ANNONSERA,...,"[C, programmeringsspråk, C++ Builder, programm...","[Engelska, Svenska]",Systemutvecklare/Programmerare,Data,Eftergymnasial utbildning två år eller längre,"[C#, programmeringsspråk, CANalyzer, analysver...",None,None,None,None


In [16]:
# Delete key/value-pairs of (must_have & nice_to_have) concept_id and legacy_ams_taxonomy_id from all objects
# Also deletes some key/value-pairs from application_contact ("email", "telephone", "contact_type")
def remove_fields(obj):
    if isinstance(obj, dict):
        keys_to_delete = []
        for key in obj.keys():
            if key in ["concept_id", "legacy_ams_taxonomy_id", "weight", "email", "telephone", "contact_type"]:
                keys_to_delete.append(key)
            else:
                remove_fields(obj[key])
        for key in keys_to_delete:
            del obj[key]
    elif isinstance(obj, list):
        for item in obj:
            remove_fields(item)

remove_fields(data)

In [17]:
# Flatten the data where there are several layers (dictionary/array)
for obj in data:
    new_obj = {}
    for key, value in obj.items():
        if isinstance(value, list) and value == []:
            new_obj[key] = None
        elif key == "workplace_address.coordinates":
            new_obj[key] = value
            continue
        elif isinstance(value, list):
            new_values = []
            for item in value:
                if isinstance(item, dict) and 'label' in item:
                    new_values.append(item['label'])
                elif item is None:
                    new_values.append(None)
            new_obj[key] = new_values[0] if len(new_values) == 1 else new_values
        elif isinstance(value, dict):
            for sub_key, sub_value in value.items():
                new_obj[f"{key}_{sub_key}"] = sub_value['label'] if sub_value is not None and 'label' in sub_value else None
        else:
            new_obj[key] = value

    obj.clear()
    obj.update(new_obj)

In [18]:
# Rensa ut alla kolumner utan de som finns angivna i "keys_to_keep"
keys_to_keep = [
    "id",
    "webpage_url",
    "headline",
    "salary_description",
    "application_deadline",
    "experience_required",
    "driving_license_required",
    "application_contacts",
    "publication_date",
    "source_type",
    "description.text",
    "description.text_formatted",
    "description.conditions",
    "employment_type.label",
    "salary_type.label",
    "duration.label",
    "working_hours_type.label",
    "scope_of_work.min",
    "scope_of_work.max",
    "employer.url",
    "employer.organization_number",
    "employer.name",
    "employer.workplace",
    "application_details.url",
    "occupation.label",
    "occupation_group.label",
    "occupation_field.label",
    "workplace_address.municipality",
    "workplace_address.region",
    "workplace_address.country",
    "workplace_address.street_address",
    "workplace_address.postcode",
    "workplace_address.city",
    "workplace_address.coordinates",
    "must_have.skills",
    "must_have.languages",
    "must_have.work_experiences",
    "must_have.education",
    "must_have.education_level",
    "nice_to_have.skills",
    "nice_to_have.languages",
    "nice_to_have.work_experiences",
    "nice_to_have.education",
    "nice_to_have.education_level"
]

for obj in data:
    new_obj = {}
    for key, value in obj.items():
        if key in keys_to_keep:
            new_obj[key] = value

    obj.clear()
    obj.update(new_obj)

In [20]:
# Save the filtered data to a new JSON file
with open('2023_FLATLOWCAL.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)